# Study for jitter effect in Energy resolution (>=0.7.0)

for xifusim-simulated files under different:

* Sampling rates (s1=156250, s2=s1/2, s4=s1/4)
* TDM configurations

- Libraries for test pulses reconstruction are always jitter+noise (as real pulses are)
- Curves plot (to show dependance of the energy error with arrival phase) and polynomial surfaces (to define the dependance inputEnergy <-> reconstructedEnergy <-> arrivalPhase) can be defined from noisy (5000 pulses) or nonoise (1000) pulses samples.
- It is observed that when the fitting is done with nonoise curves, the final FWHM vs. ArrivalPhase plots show estructure (are a bit curved), behaviour which is not observed when polynomial curves are created with samples of noisy pulses (no matter if the no-noise pulses are 1000 or 5000).



 For surface calibration, evt files are (usually) previously created with runCompareACDC.sh and calibrated here (test files are those to be calibrated and calibration files are those used for the gainScale curve also).
 - Simulated files (5000p & test files)
 - Reconstructed files
 - Surface creation -> polyfit -> coefficients
 - Calibration of "test" files

In [55]:
# ----IMPORT MODULES --------------
from astropy.io import fits
import math
import sys, os
import shutil, tempfile, shlex
import re
import warnings
import numpy as np
from sixtevars import XMLfll
from sklearn.metrics import mean_squared_error
from numpy.polynomial import Polynomial
from gainScale2DFit import fit_2D_gain_scale, polyfit2d
from normalize_in_interval import normalize_in_interval
from commands import run_comm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm, colors
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.style as mplstyle
import heasoftpy as hsp
import ipywidgets as widgets
%matplotlib widget

mplstyle.use('fast')

cwd = os.getcwd()
tmpDir = tempfile.mkdtemp()
os.environ["PFILES"] = tmpDir + ":" + os.environ["PFILES"]
os.environ["HEADASNOQUERY"] = ""
os.environ["HEADASPROMPT"] = "/dev/null/"


## Define variables 

In [ ]:
#      Simulation options to calculate jitter correction
# ========================================================
pixel="LPA2.5a"
dre = "fll"
smprt = "" # "" or "samprate2" or "samprate4"
npulsesLib = 20000 # number of pulses for libraries
npulsesGain = 5000 # number of pulses for gainScale
preBufferSize = 0
domain="T"  # or "F"
recStr="OPTFILT" #, "I2RFITTED"
lDir = "ADC"
if recStr == "I2R" or recStr == "I2RFITTED" or recStr == "I2RNOL":
    lDir = "R"
pulseLength=8192 # 8192 or shorter lengths
ofLength=8192 # 8192 for 0-padding or shorter filters otherwise
pB=0 # or >0 for using preBuffer to construct filters
noiseForSFC = "" # "" for noisy files and "_nonoise" for nonoise files 
if noiseForSFC == "_nonoise": 
    SFCpulses = 1000 # number of pulses for curve/surface creation (nonoise)
    noiseStr="NO NOISE"
if noiseForSFC == "":
    SFCpulses = 5000 # number of pulses for curve/surface creation (noisy)
    noiseStr="NOISY"
    
TSpulses = 5000 # number of pulses for test of FWHM (noisy)
#TSpulses = "2E5" # number of pulses for test of FWHM (noisy)

eurecaDir = "/dataj6/ceballos/INSTRUMEN/EURECA"
sixteDir = eurecaDir + "/testHarness/simulations/SIXTE"
sixteInst = os.environ["SIXTE"] + "/share/sixte/instruments/athena-xifu/"
pairsDir = eurecaDir + "/ERESOL/PAIRS"
impDir = pairsDir + "/PIXIMPACT"
ereDir = pairsDir + "/eresol" + pixel
gainDir = ereDir + "/gainScale"
tesDir = pairsDir + "/xifusim" + pixel
libDir = sixteDir + "/LIBRARIES/xifusim" + pixel + "/GLOBAL/" + lDir 

xmlfile = XMLfll
eners_keV = (0.2, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)  # for surface fitting
calibEnergies = [0.2, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

eners_keV = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12)  # for surface fitting
calibEnergies = [1, 2, 3, 4, 5, 6, 7, 8,9, 10, 11, 12]

libEnergies = (6,) # optimal filter in library
test_keV = (1,2,3,4,5,6,7,8,9,10,11,12) # for testing
#test_keV = (7,)

# samprate
smprtStr = ""
if smprt == "samprate2":
    smprtStr = "_samprate2"
    samprate = 78125. # Hz
    nSamples = 4096
    largeFilter = 4096
    nSgms = 4 #6
    samplesUp = 2 #2
    samplesDown = 4 #3 
    singleSeparation = 20000
    #TSexpos=540 # for the test-noisy pulses
    #NNexpos=269 # for the nonoise pulses
elif smprt == "samprate4":
    smprtStr = "_samprate4"
    samprate = 39062.5 # Hz
    nSamples = 2048
    largeFilter = 2048
    nSgms = 4 #6
    samplesUp = 2 #2
    samplesDown = 4 #3 
    singleSeparation = 10000
else:
    samprate = 156250. # Hz
    nSamples = 8192
    largeFilter = 8192
    nSgms = 3.5
    samplesUp = 3
    samplesDown = 4
    singleSeparation = 40000
    #TSexpos=525 # for the test-noisy pulses
    #NNexpos=262 # for the nonoise pulses

triggerSizeTC = preBufferSize + 2*singleSeparation + preBufferSize
triggerSizeTS = preBufferSize + pulseLength + preBufferSize
triggerTS3val = triggerSizeTS - preBufferSize

# DRE
dreStr = "_" + dre

# preBuffer (for reconstruction)
pBstr = ""
if pB > 0:
    pBstr = "_pB" + str(pB)

# polynomial fits files
coeffsfile = ereDir + "/coeffs_polyfit.dat"
surfaceFile = ereDir + "/coeffs_poly2Dfit_pL" + str(pulseLength) + "_" + recStr + str(ofLength) + pBstr + dreStr + ".dat"

# define optimized colors for offset plots
cols = [(165,0,38),(215,48,39),(244,109,67),(253,174,97),(254,224,144),(224,243,248),(171,217,233),(116,173,209),
        (69,117,180),(49,54,149)]
reversed_cols = cols[::-1]
for i in range(0,len(cols)):
        cols[i] = tuple(round((x/255.),2) for x in reversed_cols[i])
        # print("Color=", cols[i])



## ... and filenames

In [ ]:
# a) Filenames for surface fitting and dependency curves 
# ========================================================
evtFs = list()
calibFs = list()
tesFs = list()
PL = max(pulseLength, ofLength)
libStr = "_fixedlib6OF_"
libfile = (libDir + "/library6keV_PL" + str(PL) + "_" + str(npulsesLib) + "p" + smprtStr + noiseForSFC + pBstr + dreStr + ".fits")

for i in range(0,len(eners_keV)):
    
    # xifusim-simulated file
    file = (tesDir + "/sep" + str(singleSeparation) + "sam_" + str(SFCpulses) + "p_" + str(eners_keV[i]) +
            "keV" + smprtStr + noiseForSFC + dreStr + ".fits")
    tesFs.append(file)
    
    # reconstructed (uncalibrated) file
    file = (gainDir + "/events_sep" + str(singleSeparation) + "sam_" + str(SFCpulses) + "p_SIRENA" +
            str(nSamples)+ "_pL" + str(pulseLength) + "_" + str(eners_keV[i]) +
            "keV_STC_" + domain + libStr + recStr + str(ofLength) + pBstr + smprtStr + noiseForSFC + dreStr + ".fits")
    evtFs.append(file)

    # reconstructed AND 1D-calibrated files
    file = (ereDir + "/events_sep" + str(singleSeparation) + "sam_" + str(SFCpulses) + "p_SIRENA" +
            str(nSamples)+ "_pL" + str(pulseLength) + "_" + str(eners_keV[i]) +
            "keV_STC_" + domain + libStr + recStr + str(ofLength) + pBstr + smprtStr + noiseForSFC + dreStr + "_HR.calib1D")
    calibFs.append(file)

# a) Filenames for test (noisy)
# =============================
#test_impFs = list()
test_tesFs = list()
test_evtFs = list()
test_calibFs = list()
tnpulsesLib = 20000
    
tlibfile = (sixteDir + "/LIBRARIES/xifusimLPA75um/GLOBAL/" + libDir + "/library6keV_PL" + str(PL) + "_" +
           str(tnpulsesLib) + "p" + smprtStr + dreStr + ".fits")

for i in range(len(test_keV)):
    file = (tesDir + "/sep" + str(singleSeparation) + "sam_" + str(TSpulses) + "p_" +
            str(test_keV[i]) + "keV" + smprtStr + dreStr + ".fits")
    test_tesFs.append(file)
    file = (ereDir + "/events_sep" + str(singleSeparation) + "sam_" + str(TSpulses) +
            "p_SIRENA" + str(nSamples)+ "_pL" + str(pulseLength) + "_" + str(test_keV[i]) +
            "keV_STC_" + domain + libStr + recStr + str(ofLength) + pBstr + smprtStr + 
            dreStr + ".fits")
    test_evtFs.append(file)
    file = (ereDir + "/events_sep" + str(singleSeparation) + "sam_" + str(TSpulses) +
            "p_SIRENA" + str(nSamples)+ "_pL" + str(pulseLength) + "_" + str(test_keV[i]) +
            "keV_STC_" + domain + libStr + recStr + str(ofLength) + pBstr + smprtStr +
            dreStr + "_HR.calib1D")
    test_calibFs.append(file)



## (Optional) RECONSTRUCT Data files for surface fitting (those from gainscale)

PIXIMPACT -> XIFUSIM (serpens)-> SIRENA(STC) -> CALIBRATED


In [ ]:
run = 0
# Run SIRENA to reconstruct calibration files
# ===========================================
if run:
    print("Reconstructing SURFACE",SFCpulses, "single files with lib:", libfile)
    print("DRE=",dre,"samprate=",smprtStr)
    #sys.exit()
    for i in range(0,len(eners_keV)):
        comm = ("tesreconstruction Recordfile=" + tesFs[i] + " TesEventFile=" + evtFs[i] + " Rcmethod=SIRENA" + 
                " PulseLength="+ str(pulseLength) + " LibraryFile=" + libfile +
                " samplesUp=" + str(samplesUp) + " nSgms=" + str(nSgms) + " samplesDown=" + str(samplesDown) + 
                " opmode=1 OFLib=yes FilterDomain=" + domain + " detectionMode=STC" +
                " FilterMethod=F0 clobber=yes EnergyMethod=" + recStr + " LagsOrNot=1" + 
                " XMLFile=" + xmlfile + " filtEeV=6000.0 OFStrategy=FIXED OFLength=" + str(ofLength) + 
                " preBuffer=" + str(pB))
        mess = "Reconstructing: " + tesFs[i]
        run_comm(comm, mess)
    print("########################")        
    print("SIRENA files created")            
    print("########################")        
#sys.exit()

In [ ]:
run = 0
# Correct energies from gain scale
#======================================
if run: 
    print("Correcting SURFACE",SFCpulses, "pulses single files with lib:", libfile)
    print("DRE=",dre,"samprate=",smprtStr)
    for i in range(0,len(eners_keV)):
        alias = "pL" + str(pulseLength) + "_STC_T_fixedlib6OF_" + recStr  + str(ofLength) + smprtStr + \
        noiseForSFC + dreStr 
        print("alias=", alias)
        comm = ("python calibEnergies.py --inFile=" + evtFs[i] + " --outFile=" + calibFs[i] + 
                " --coeffsFile=" + coeffsfile + " --alias=" + alias)
        mess = "Calibrating: " + evtFs[i]
        run_comm(comm, mess)

    print("########################")        
    print("CALIBRATED files created")            
    print("########################")        
#sys.exit()

## (Optional) RECONSTRUCT Test Files 
PIXIMPACT -> XIFUSIM (serpens)-> SIRENA(STC) -> CALIBRATED


In [ ]:
run = 0
# Run SIRENA over test files
# ==========================
if run:
    print("Creating SIRENA test files for DRE=",dre,"SAMPRATE=",smprt,"and libfile=", tlibfile)
    for i in range(0,len(test_keV)):
        comm = ("tesreconstruction Recordfile=" + test_tesFs[i] + " TesEventFile=" + test_evtFs[i] +
                " PulseLength=" + str(pulseLength) + " LibraryFile=" + tlibfile +
                " samplesUp=" + str(samplesUp) + " nSgms=" + str(nSgms) + " samplesDown=" + str(samplesDown) + 
                " opmode=1 OFLib=yes FilterDomain=" + domain + " detectionMode=STC" +
                " FilterMethod=F0 clobber=yes EventListSize=1000 EnergyMethod=" + recStr + " LagsOrNot=1" +
                " XMLFile=" + xmlfile + " filtEeV=6000.0 OFStrategy=FIXED OFLength=" + str(ofLength)+ 
                " preBuffer=" + str(pB) )
        mess = "Reconstructing: " + test_tesFs[i]
        run_comm(comm, mess)
    print("########################")        
    print("test SIRENA files created")            
    print("########################")   

In [ ]:
run=0
if run:
    print("Creating CALIB test files for DRE,SAMPRATE",dre,smprt)
    # Correct energies from gain scale
    # ================================
    for i in range(0,len(test_keV)):
        alias = "pL" + str(pulseLength) + "_STC_T_fixedlib6OF_" + recStr  + str(ofLength) + smprtStr + dreStr 
        comm = ("python calibEnergies.py --inFile=" + test_evtFs[i] + " --outFile=" + test_calibFs[i] + 
                " --coeffsFile=" + coeffsfile + " --alias=" + alias)
        mess = "Calibrating: " + test_evtFs[i]
        run_comm(comm, mess)
    print("##################################")
    print("TEST Files created and calibrated")
    print("##################################")

## Define SURFACE for fitting
A surface is defined along three axes: reconstructed PH, OFfset and input (calibration energy)

### a) Read Surface Data and Select non-0-PHI events
Events with PHI=0.0 would indicate that no parabola could be fit (multiple pulses?)
Reconstructed (calibrated and uncalibrated) data

In [21]:
# Find minimum number of total pulses AND non-0-PHI pulses
npls = list()
nn0pls = list()
for i in range(0,len(eners_keV)):
    #print ("Opening file ",evtFs[i])
    uncal = fits.open(evtFs[i], memmap=True)
    naxis2 = uncal[1].header["NAXIS2"]
    npls.append(naxis2)
    uncalTab = uncal[1].data
    uncalPHIS = uncalTab['PHI']
    non0 = np.count_nonzero(uncalPHIS)
    nn0pls.append(non0)
    uncal.close()
nCpulses = np.amin(npls)
non0ct = np.amin(nn0pls)
print("Common number of pulses=",nCpulses)
print("Common number of non-0-phi pulses=",non0ct)
print("PHI=0 events:",nCpulses-non0ct)

Common number of pulses= 5024
Common umber of non-0-phi pulses= 5024
PHI=0 events: 0


In [22]:
# Read data in calibration (gainScale) files
ph_values_non0 = np.zeros((len(eners_keV),non0ct))
phcal_values_non0 = np.zeros((len(eners_keV),non0ct))
PHI_values_non0 = np.zeros((len(eners_keV),non0ct))
PHIn_values_non0 = np.zeros((len(eners_keV),non0ct))

for i in range(0,len(eners_keV)):
    print("Reading files for calibration Energy=", eners_keV[i])
    # Detected events (uncalibrated)
    uncal = fits.open(evtFs[i], memmap=True)
    uncalTab = uncal[1].data
    uncalTimes = uncalTab['TIME']
    uncalEners = uncalTab['SIGNAL']
    uncalPHIS = uncalTab['PHI']
    uncalLAGS = uncalTab['LAGS']
    non0indices = np.nonzero(uncalPHIS)[0:non0ct]
    
    ph_values_non0[i,] = uncalEners[non0indices]
    PHI_values_non0[i,] = uncalPHIS[non0indices]
    PHIn_values_non0[i,] = uncalPHIS[non0indices] + uncalLAGS[non0indices] 
    uncal.close()
    
    # Calibrated events (gain-scale corrected)
    cal = fits.open(calibFs[i], memmap=True)
    calTab = cal[1].data
    calTimes = calTab['TIME']
    calEners = calTab['SIGNAL']
    phcal_values_non0[i,] = calEners[non0indices]
    cal.close()

Reading files for calibration Energy= 1
Reading files for calibration Energy= 2
Reading files for calibration Energy= 3
Reading files for calibration Energy= 4
Reading files for calibration Energy= 5
Reading files for calibration Energy= 6
Reading files for calibration Energy= 7
Reading files for calibration Energy= 8
Reading files for calibration Energy= 9
Reading files for calibration Energy= 10
Reading files for calibration Energy= 11
Reading files for calibration Energy= 12


### b) Plot curves
First, initial curves are plotted:
    - Relative Error in Calibrated energy vs Offset
    - Relative Error in UNCalibrated energy vs Offset

In [25]:
plot=1
if plot:
    fig = plt.figure(figsize=(8,6))

    # Plot RelativeErrCorrEner vs PHI measured Phases
    # =================================================
    ax = fig.add_subplot(221)
    ax.set_ylabel("Rel. Ener estimation error (%)\n (gain scale corrected)")
    for i in range(len(eners_keV)):
        inkeV = eners_keV[i]
        relErr = (phcal_values_non0[i,] - inkeV)*100./inkeV  # [%] Relative Energy Estimation Error
        labelLeg = "E=" + str(inkeV) + " keV"
        #fc = cols[i]
        #ax.plot(PHI_values[i,], relErr, marker='o', markerfacecolor=fc, linestyle='none', 
        #         markeredgecolor=cols[i], label=labelLeg, ms=3)
        ax.plot(PHI_values_non0[i,], relErr, marker='o', linestyle='none', label=labelLeg, ms=3)
        ax.axvline(x=0, linestyle="dashed",color="gray")
        ax.legend(numpoints=3,prop={'size': 6})

    # Plot RelativeErrCorrEner vs Measured (PHI+n) Phases
    # =================================================
    ax = fig.add_subplot(222)
    ax.set_ylabel("Rel Ener estimation error (%)\n (gain scale corrected)")
    for i in range(len(eners_keV)):
        inkeV = eners_keV[i]
        relErr = (phcal_values_non0[i,] - inkeV)*100./inkeV  # [%] Relative Energy Estimation Error
        labelLeg = "E=" + str(inkeV) + " keV"
        #fc = cols[i]
        #ax.plot(PHIn_values[i,], relErr, marker='o', markerfacecolor=fc, linestyle='none', 
        #         markeredgecolor=cols[i], label=labelLeg, ms=3)
        ax.plot(PHIn_values_non0[i,], relErr, marker='o', linestyle='none', label=labelLeg, ms=3)
        ax.axvline(x=0, linestyle="dashed",color="gray")
        ax.legend(numpoints=3,prop={'size': 6})

        minErr = min(relErr)
        maxErr = max(relErr)
        #print("E,minErr,maxErr, Diff=", eners_keV[i], minErr, maxErr, abs(maxErr-minErr))

    # Plot RelativeErrUncorrEner vs Measured (PHI) Phases
    # ====================================================
    ax = fig.add_subplot(223)
    ax.set_xlabel("Measured (PHI) arrival phase")
    ax.set_ylabel("Relative Energy estimation error (%)\n (uncalibrated)")
    for i in range(len(eners_keV)):
        inkeV = eners_keV[i]
        relErr = (ph_values_non0[i,] - inkeV)*100./inkeV  # [%] Relative Energy Estimation Error
        labelLeg = "E=" + str(inkeV) + " keV"
        #fc = cols[i]
        #ax.plot(PHI_values[i,], relErr, marker='o', markerfacecolor=fc, linestyle='none', 
        #         markeredgecolor=cols[i], label=labelLeg, ms=3)
        ax.plot(PHI_values_non0[i,], relErr, marker='o', linestyle='none', label=labelLeg, ms=3)
        ax.axvline(x=0, linestyle="dashed",color="gray")
        ax.legend(numpoints=3,prop={'size': 6})

    # Plot RelativeErrUnCorrEner vs measures (PHI+n) phases
    # =================================================
    ax = fig.add_subplot(224)
    ax.set_xlabel("Measured (PHI+n) arrival phase")
    ax.set_ylabel("Relative Energy estimation error (%)\n (uncalibrated)")
    for i in range(len(eners_keV)):
        inkeV = eners_keV[i]
        relErr = (ph_values_non0[i,] - inkeV)*100./inkeV  # [%] Relative Energy Estimation Error
        labelLeg = "E=" + str(inkeV) + " keV"
        #fc = cols[i]
        #ax.plot(PHIn_values[i,], relErr, marker='o', markerfacecolor=fc, linestyle='none', 
        #         markeredgecolor=cols[i], label=labelLeg, ms=3)
        ax.plot(PHIn_values_non0[i,], relErr, marker='o', linestyle='none', label=labelLeg, ms=3)
        ax.axvline(x=0, linestyle="dashed",color="gray")
        ax.legend(numpoints=3,prop={'size': 6})

    fileplot = (ereDir + "/EreconFLL" + smprtStr + "_SFC" + str(SFCpulses) + "p" + noiseForSFC + "_" +
                str(TSpulses) + "p_pL" + str(pulseLength) + "_" + recStr + str(ofLength) + pBstr + "_offsets.png")
    fig.savefig(fileplot)    
    fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

###  c) Plot 3D surfaces 
3D visualization: 
    - (relative err in calib energy) vs Offset vs (input energy)
    - (uncalibrated reconstructed PH) vs offset vs (input energy): surface to be fitted

In [27]:
plot=1
if plot:
    plt.close()
    print("Plotting 3D surfaces")
    X = np.zeros((len(eners_keV), non0ct))
    Y = np.zeros((len(eners_keV), non0ct))
    YS = np.zeros((len(eners_keV), non0ct))
    Z = np.zeros((len(eners_keV), non0ct))

    # PLOT      EcalibError  vs. offset  vs. Einput
    # ==============================================
    fig = plt.figure(figsize=(10,5))
    suptitle='Error in (cal)Energy estimation for samprate=' + str(samprate)
    fig.suptitle(suptitle, fontsize=14, fontweight='bold')

    #ax = fig.add_subplot(131, projection='3d')
    ax = fig.add_subplot(121, projection='3d')
    ax.set_xlabel('PHI (arrival phase)')
    ax.set_ylabel('Rel. E estim (cal) %')
    ax.set_zlabel('Input energy (keV)')
    for i in range(0,len(eners_keV)):
        X[i,] = PHI_values_non0[i,]
        Y[i,] = (phcal_values_non0[i,]-eners_keV[i])*100./eners_keV[i]
        Z[i,] = np.repeat(eners_keV[i], non0ct)
        #XS[i,] = np.sort(X[i,],0)
        YS[i,] = [x for _,x in sorted(zip(X[i,],Y[i,]))]
        #ax.scatter(xs=X[i,], ys=Y[i,], zs=eners_keV[i], zdir='z', s=20, c=cols[i], depthshade=True)
        ax.scatter(xs=X[i,], ys=Y[i,], zs=eners_keV[i], zdir='z', s=20, depthshade=True)
    XS = np.sort(X,1)

    #ax2 = fig.add_subplot(132, projection='3d')
    #ax2.plot_wireframe(X=XS, Y=YS, Z=Z)
    #ax2.set_xlabel('PHI (arrival phase)')
    #ax2.set_ylabel('Rel. E estim (cal) %')
    #ax2.set_zlabel('Input energy (keV)')

    #ax3 = fig.add_subplot(133, projection='3d')
    ax3 = fig.add_subplot(122, projection='3d')
    surf = ax3.plot_surface(X=XS, Y=YS, Z=Z, cmap=cm.coolwarm, antialiased=True)
    ax3.set_xlabel('PHI (arrival phase)')
    ax3.set_ylabel('Rel. E estim (cal) %')
    ax3.set_zlabel('Input energy (keV)')
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.tight_layout()

    
    # PLOT      Euncalib  vs. offset  vs. Einput
    # ===============================================
    fig = plt.figure(figsize=(10,5))
    fig.suptitle('(Uncalibrated) reconstructed PH', fontsize=14, fontweight='bold')

    #ax = fig.add_subplot(131, projection='3d')
    ax = fig.add_subplot(121, projection='3d')
    ax.set_xlabel('PHI (arrival phase)')
    ax.set_ylabel('Recons PH')
    ax.set_zlabel('Input energy (keV)')
    for i in range(0,len(eners_keV)):
        Y[i,] = ph_values_non0[i,]
        YS[i,] = [x for _,x in sorted(zip(X[i,],Y[i,]))]
        #ax.scatter(xs=X[i,], ys=Y[i,], zs=eners_keV[i], zdir='z', s=20, c=cols[i], depthshade=True)
        ax.scatter(xs=X[i,], ys=Y[i,], zs=eners_keV[i], zdir='z', s=20, depthshade=True)
    XS = np.sort(X,1)

    #ax2 = fig.add_subplot(132, projection='3d')
    #ax2.plot_wireframe(X=XS, Y=YS, Z=Z)
    #ax2.set_xlabel('PHI (arrival phase)')
    #ax2.set_ylabel('Uncal E (keV)')
    #ax2.set_zlabel('Input energy (keV)')

    #ax3 = fig.add_subplot(133, projection='3d')
    ax3 = fig.add_subplot(122, projection='3d')
    surf = ax3.plot_surface(X=XS, Y=YS, Z=Z, cmap=cm.coolwarm, antialiased=True)
    ax3.set_xlabel('PHI (arrival phase)')
    ax3.set_ylabel('Recons PH')
    ax3.set_zlabel('Input energy (keV)')
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.tight_layout()


Plotting 3D surfaces


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Fitting of a 2D polynomial to PH (reconstructed-uncalib-PH) vs Offsets

Based on PP functions. Functions input:

* Reconstructed Pulse Heights (ph_values): PH are reconstructed with a 20000p library + not corrected/calibrated

* Arrival Phases (phase_values)

* Calibration Energies (energy_list)

### a) Decide best degree of surface, based on residuals of fitting surface over calibration files

In [71]:
#plot residuals of correction for different degree(s) of fitting surface
plt.close()
fig = plt.figure(figsize=(12,10))
ii=0
nrows=4
ncols=3
mindeg=2
maxdeg=6
cmap = plt.cm.get_cmap('Dark2')     

for i in range(0,len(eners_keV)):  
    ii+=1
    ax = fig.add_subplot(nrows,ncols,ii)
    for deg in range(mindeg,maxdeg+1):
        coeffs=fit_2D_gain_scale(ph_values_non0, PHI_values_non0, eners_keV, deg=deg, show=True)
        # Arrival Phases & Corrected energies
        corrE = np.polynomial.polynomial.polyval2d(ph_values_non0[i,],PHI_values_non0[i,],coeffs)
        lab = "deg="+str(deg)
        color = 'C'+str(deg-mindeg)
        #norm = colors.Normalize(vmin=mindeg, vmax=maxdeg)
        #color = cmap(norm(deg))
        #color=cmap(deg-mindeg)
        #print(norm(deg))
        ax.plot(PHI_values_non0[i,], corrE-eners_keV[i], marker='o', linestyle='none', ms=1, alpha=0.05,color=color)
        
        fpol = Polynomial.fit(PHI_values_non0[i,],corrE-eners_keV[i],2)
        xs = np.linspace(np.min(PHI_values_non0[i,]),np.max(PHI_values_non0[i,]),50)
        #ax.plot(xs,fpol(xs), linestyle='-', color='white', linewidth=4)
        ax.plot(xs,fpol(xs), linestyle='-',color=color,label=lab)
        
        ax.legend(prop={'size': 6})
        
        
    tit="E="+str(eners_keV[i])+"keV"
    ax.title.set_text(tit)
    ax.set_xlabel("PHI (arrival phase)")
    ax.set_ylabel("Err in CorrEnerg (keV)")
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### b) Check numerical values of some corrections

In [74]:
BESTDEG=4

print("Do & Test 2D fitting of test files")   
# with total PHI+LAGS
#coeffs=fit_2D_gain_scale(ph_values_non0, PHIn_values_non0, eners_keV, deg=6, show=True)

BESTcoeffs=fit_2D_gain_scale(ph_values_non0, PHI_values_non0, eners_keV, deg=BESTDEG, show=True)

print("Checking a particular pulse of every calibration energy")
print("Einput  PH       PHIn_val   ph_cal(1D) E_corr(poly)")
phnum=10
for i in range(len(eners_keV)):
    corrE = np.polynomial.polynomial.polyval2d(ph_values_non0[i,phnum],PHI_values_non0[i,phnum],coeffs)
    print('{:2d}'.format(eners_keV[i]),"    ", 
          '{:0.4f}'.format(ph_values_non0[i,phnum]), "   ", 
          '{:+0.4f}'.format(PHIn_values_non0[i,phnum]),"    ", 
          '{:0.4f}'.format(phcal_values_non0[i,phnum]),"   ", 
          '{:0.4f}'.format(corrE))

np.savetxt(surfaceFile, coeffs, header='Polynomial surface coefficients. Use as: \
    calibEnergy=np.polynomial.polynomial.polyval2d(pulse_height,offsetN,np.array(coeffs))')


Do & Test 2D fitting of test files
Checking a particular pulse of every calibration energy
Einput  PH       PHIn_val   ph_cal(1D) E_corr(poly)
 1      1.1066     +0.0226      1.0038     1.0001
 2      2.1680     +0.0087      2.0060     2.0013
 3      3.1842     -0.0006      3.0009     3.0003
 4      4.1616     -0.0120      3.9965     4.0006
 5      5.1003     -0.0180      4.9954     5.0006
 6      5.9999     -0.0010      5.9997     5.9998
 7      6.8508     +0.1085      7.0016     6.9983
 8      7.6535     +0.2247      8.0040     8.0009
 9      8.4006     +0.3195      8.9998     8.9990
10      9.0968     +0.4034      9.9974     9.9988
11      9.7405     +0.4829      10.9982     11.0000
12      10.3277     +0.5874      11.9999     11.9993


## Test files energy corrections

Compare resconstructed energies from 1keV to 12 keV before and after corrections.
For this, we use *TSpulses* reconstructed with a *jitter_noisy* library using the coefficients calculated above.

In [76]:
# plot corrections for all the calibration energies
plot = 1
ii=0
if plot:
    print("Correcting energies for ", TSpulses, "pulses with DRE=", dre, "and samprate=",samprate)

    fig = plt.figure(figsize=(9,30))
    irws = len(test_keV)
    #irws=3
    ncols=3
    nrows=irws

    for i in range(0,irws):
        # Detected uncal events
        print("Reading ", test_evtFs[i])
        uncal = fits.open(test_evtFs[i], memmap=True)
        uncalTab = uncal[1].data
        arrPhase = uncalTab['PHI']
        # arrival phase to (-0.5,0.5) interval
        #arrPhase = uncalTab['PHI'] - 1.*np.trunc(uncalTab['PHI']/0.5)
        # taking into account LAGS
        #arrPhase = uncalTab['PHI'] + uncalTab['LAGS']
        arrPhase = arrPhase[np.nonzero(arrPhase)]
        uncalEners = uncalTab['SIGNAL']
        uncalEners = uncalEners[np.nonzero(arrPhase)]
        uncal.close()

        # Detected cal events
        cal = fits.open(test_calibFs[i], memmap=True)
        calTab = cal[1].data
        calEners = calTab['SIGNAL']
        cal.close()

        # Arrival Phases & Corrected energies
        corrE = np.polynomial.polynomial.polyval2d(uncalEners, arrPhase,BESTcoeffs)

        # FWHM calculations
        fwhmUnCal = 2.35*np.std(uncalEners)*1000# ~eV
        fwhmUnCalStr = 'FWHM={:0.3f}~eV'.format(fwhmUnCal)
        print(fwhmUnCal,fwhmUnCalStr)
        
        fwhmCal = 2.35*np.std(calEners)*1000 # eV
        fwhmCal_err = fwhmCal/np.sqrt(2*len(calEners)-2) # eV
        fwhmCalStr = 'FWHM={:0.3f}+/-{:0.3f} eV'.format(fwhmCal, fwhmCal_err) # eV

        fwhmCorr = 2.35*np.std(corrE)*1000. # eV
        fwhmCorr_err = fwhmCorr/np.sqrt(2*len(corrE)-2)
        fwhmCorrStr = 'FWHM={:0.3f}+/-{:0.3f} eV'.format(fwhmCorr, fwhmCorr_err) # eV

        #--------------------------- PLOTTING ----------------------------------------#

        # Plot UnCalE vs ArrPhse 
        # ============================
        ii+=1
        #figstr = str(irws)+"3"+str(3*i+1)
        #ax=plt.subplot2grid((irws,3),(i,0))
        ax = fig.add_subplot(nrows,ncols,ii)
        if ii == nrows*3-2:
            ax.set_xlabel("PHI (arrival phase)")
        ax.set_ylabel("Recons PH")
        ax.plot(arrPhase, uncalEners, marker='o', linestyle='none', ms=3)
        ax.axhline(np.mean(uncalEners),linestyle='-', color='grey', linewidth=1)
        ax.text(0.95, 0.01, fwhmUnCalStr,verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes, color='green', fontsize=8)

        # Plot CalE vs ArrPhse 
        # ============================
        ii+=1
        #figstr = str(irws)+"3"+str(3*i+2)
        #ax=plt.subplot2grid((irws,3),(i,1))
        ax = fig.add_subplot(nrows,ncols,ii)
        if ii == nrows*3-1:
            ax.set_xlabel("PHI (arrival phase)")
        ylabel="Calib Energy (keV)"
        ax.set_ylabel(ylabel)
        ax.plot(arrPhase, calEners, marker='o', linestyle='none', ms=3)
        ax.axhline(np.mean(calEners),linestyle='-', color='grey', linewidth=1)
        ax.text(0.95, 0.01, fwhmCalStr,verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes, color='green', fontsize=8)
        fpol = Polynomial.fit(arrPhase,calEners,2)
        xs = np.linspace(np.min(arrPhase),np.max(arrPhase),50)
        ax.plot(xs,fpol(xs), linestyle='-', color='white', linewidth=4)
        ax.plot(xs,fpol(xs), linestyle='-', color='green', linewidth=2)

        # Plot CorrE vs ArrPhse 
        # ============================
        figstr = str(irws)+"3"+str(3*i+3)
        ii+=1
        #ax=plt.subplot2grid((irws,3),(i,2))
        ax = fig.add_subplot(nrows,ncols,ii)
        if ii == nrows*3:
            ax.set_xlabel("PHI (arrival phase)")
        ax.set_ylabel("Corr Energy (keV)")
        ax.text(0.95, 0.01, fwhmCorrStr,verticalalignment='bottom', horizontalalignment='right',
        transform=ax.transAxes, color='green', fontsize=8)
        # plot initial polynomial fit
        #-----------------------------
        fpol = Polynomial.fit(arrPhase,corrE,2)
        #corrEcorr = test_keV[i]+corrE-fpol(arrPhase)
        #ax.plot(arrPhase, corrEcorr, marker='o', linestyle='none', ms=3)
        ax.plot(arrPhase, corrE, marker='o', linestyle='none', ms=3)
        xs = np.linspace(np.min(arrPhase),np.max(arrPhase),50)
        ax.plot(xs,fpol(xs), linestyle='-', color='white', linewidth=4)
        ax.plot(xs,fpol(xs), linestyle='-', color='red', linewidth=2)
        rms = np.sqrt(mean_squared_error(corrE,fpol(arrPhase)))

        ax.axhline(np.mean(corrE),linestyle='-', color='grey', linewidth=1)

        # plot secondary polynomial fit
        #fpol2 = Polynomial.fit(arrPhase,corrEcorr,2)
        #ax.plot(xs,fpol2(xs), linestyle='-', color='white', linewidth=4)
        #ax.plot(xs,fpol2(xs), linestyle='-', color='green', linewidth=2)
        #fwhmCorr2 = 2.35*np.std(corrEcorr)*1000. # eV
        #fwhmCorr2_err = fwhmCorr2/np.sqrt(2*-2)
        #fwhmCorr2Str = 'FWHM={:0.2f}+/-{:0.3f} eV'.format(fwhmCorr2, fwhmCorr2_err) # eV
        #xl2 = np.min(arrPhase)
        #ax.annotate(fwhmCorr2Str, xy=(xl2,yl), color="green")

    plt.tight_layout()


Correcting energies for  5000 pulses with DRE= fll and samprate= 156250.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Reading  /dataj6/ceballos/INSTRUMEN/EURECA/ERESOL/PAIRS/eresolLPA2.5a/events_sep40000sam_5000p_SIRENA8192_pL8192_1keV_STC_T_fixedlib6OF_OPTFILT8192_fll.fits
2.00274792174607 FWHM=2.003~eV
Reading  /dataj6/ceballos/INSTRUMEN/EURECA/ERESOL/PAIRS/eresolLPA2.5a/events_sep40000sam_5000p_SIRENA8192_pL8192_2keV_STC_T_fixedlib6OF_OPTFILT8192_fll.fits
1.982190151702288 FWHM=1.982~eV
Reading  /dataj6/ceballos/INSTRUMEN/EURECA/ERESOL/PAIRS/eresolLPA2.5a/events_sep40000sam_5000p_SIRENA8192_pL8192_3keV_STC_T_fixedlib6OF_OPTFILT8192_fll.fits
1.9261349663882998 FWHM=1.926~eV
Reading  /dataj6/ceballos/INSTRUMEN/EURECA/ERESOL/PAIRS/eresolLPA2.5a/events_sep40000sam_5000p_SIRENA8192_pL8192_4keV_STC_T_fixedlib6OF_OPTFILT8192_fll.fits
1.889507995116385 FWHM=1.890~eV
Reading  /dataj6/ceballos/INSTRUMEN/EURECA/ERESOL/PAIRS/eresolLPA2.5a/events_sep40000sam_5000p_SIRENA8192_pL8192_5keV_STC_T_fixedlib6OF_OPTFILT8192_fll.fits
1.8243423931110685 FWHM=1.824~eV
Reading  /dataj6/ceballos/INSTRUMEN/EURECA/ERESOL/PAIR

In [ ]:
plt.close()
# Correct and Plot alone 7 keV figure

for i in range(0,len(test_keV)):
    if test_keV[i] == 7:
        # Detected uncal events
        uncal = fits.open(test_evtFs[i], memmap=True)
        uncalTab = uncal[1].data
        #arrPhase = uncalTab['PHI'] + uncalTab['LAGS']
        arrPhase = uncalTab['PHI']
        uncalEners = uncalTab['SIGNAL']
        uncal.close()

        uncalEners_non0 = uncalEners[np.nonzero(arrPhase)]
        arrPhase_non0 = arrPhase[np.nonzero(arrPhase)]
    
corrE7 = np.polynomial.polynomial.polyval2d(uncalEners_non0, arrPhase_non0,coeffs)
fwhmCorr7 = 2.35*np.std(corrE7)*1000. # eV
fwhmCorr7_err = fwhmCorr7/np.sqrt(2*float(TSpulses)-2)
if fwhmCorr7_err < 0.01:
    fwhmCorrStr7 = 'FWHM={:0.3f}+/-{:0.3f} eV'.format(fwhmCorr7, fwhmCorr7_err) # eV
elif fwhmCorr7_err < 0.1:
    fwhmCorrStr7 = 'FWHM={:0.2f}+/-{:0.2f} eV'.format(fwhmCorr7, fwhmCorr7_err) # eV
else:
    fwhmCorrStr7 = 'FWHM={:0.1f}+/-{:0.1f} eV'.format(fwhmCorr7, fwhmCorr7_err) # eV

fpol7 = Polynomial.fit(arrPhase,corrE7,2)

fig2 = plt.figure(figsize=(5,3))
#suptitle = 'Sampling frequency={:0.2f} kHz'.format(samprate)
suptitle = "7 keV photons, " + recStr + " filter-" + str(ofLength) + "samples long"
fig2.suptitle(suptitle)
ax2 = fig2.add_subplot("111")
ax2.set_xlabel("PHI arrival phase")
ax2.set_ylabel("Corrected Energy (keV)")
ax2.text(0.95, 0.01, fwhmCorrStr7,verticalalignment='bottom', horizontalalignment='right',
        transform=ax2.transAxes, color='green', fontsize=8)
ax2.plot(arrPhase, corrE7, marker='o', linestyle='none', ms=3)
xs = np.linspace(np.min(arrPhase),np.max(arrPhase),50)
ax2.plot(xs,fpol7(xs), linestyle='-', color='white', linewidth=4)
ax2.plot(xs,fpol7(xs), linestyle='-', color='red', linewidth=2)
fileplot2 = (ereDir+ "/EreconBBFB"+ smprtStr + "_SFC" + str(SFCpulses) + "p" + noiseForSFC + "_" +
            str(TSpulses) + "p_pL" + str(pulseLength) + "_" + recStr + str(ofLength) + pBstr + "_7keV.png")
fig2.savefig(fileplot2)
plt.tight_layout()